# SW-11-RDFStar

**Navigation** : [<< 10-JSONLD](SW-10-JSONLD.ipynb) | [Index](README.md) | [12-KnowledgeGraphs >>](SW-12-KnowledgeGraphs.ipynb)

## RDF 1.2 (RDF-Star) : Assertions sur des Assertions

### Duree estimee : 40 minutes

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre la motivation de RDF 1.2 (anciennement RDF-Star) et le probleme de la reification classique
2. Creer des **quoted triples** (triplets cites) pour annoter des assertions
3. Manipuler des triplets imbriques avec rdflib 7.x
4. Interroger des quoted triples avec SPARQL-Star
5. Appliquer RDF-Star a des cas concrets : provenance, confiance, annotations temporelles

### Concepts cles

| Concept | Description |
|---------|-------------|
| RDF 1.2 | Evolution du standard RDF integrant les quoted triples (W3C, 2024) |
| Quoted Triple | Triplet utilise comme sujet ou objet d'un autre triplet : `<< s p o >>` |
| Reification classique | Mecanisme RDF 1.0 pour parler de triplets (4 triplets par assertion) |
| SPARQL-Star | Extension de SPARQL pour interroger les quoted triples |
| Provenance | Qui a dit quoi ? Source et fiabilite d'une assertion |

### Prerequis
- Python 3.10+
- Notebook [SW-8-PythonRDF](SW-8-PythonRDF.ipynb) (bases rdflib)
- rdflib >= 7.0 (support RDF-Star)

---

## Installation des dependances

In [ ]:
%pip install -q rdflib

Verifions que rdflib est en version 7.x ou superieure, necessaire pour le support RDF-Star.

In [ ]:
import rdflib

version = rdflib.__version__
major = int(version.split(".")[0])

print(f"rdflib version : {version}")
if major >= 7:
    print("Support RDF-Star disponible.")
else:
    print(f"ATTENTION : rdflib {version} ne supporte pas RDF-Star. Version 7.x requise.")

---

## 1. Le probleme : pourquoi parler de triplets ?

RDF permet de representer des faits sous forme de triplets. Mais que faire quand on veut **parler d'un triplet lui-meme** ?

Exemples de besoins courants :

| Besoin | Exemple | Ce qu'on veut exprimer |
|--------|---------|------------------------|
| **Provenance** | Source d'une information | "Wikipedia dit que Paris est la capitale de la France" |
| **Confiance** | Degre de certitude | "L'assertion 'X connait Y' a un score de confiance de 0.85" |
| **Temporalite** | Validite dans le temps | "Bob travaille chez Acme depuis 2020" |
| **Attribution** | Qui affirme quoi | "Alice affirme que Bob connait Carol" |
| **Annotation** | Metadata sur un fait | "Ce lien a ete decouvert par le crawler v2" |

En RDF classique (1.0/1.1), le seul mecanisme disponible est la **reification**, qui s'avere lourde et peu pratique.

### 1.1 La reification classique : 4 triplets pour 1 assertion

Pour dire "Alice dit que Bob connait Carol" en RDF classique, il faut creer une ressource intermediaire (un `rdf:Statement`) et decomposer l'assertion en 4 triplets :

```turtle
# Le triplet original qu'on veut annoter :
#   ex:Bob foaf:knows ex:Carol .

# Reification classique (4 triplets supplementaires) :
ex:statement1 rdf:type rdf:Statement .
ex:statement1 rdf:subject ex:Bob .
ex:statement1 rdf:predicate foaf:knows .
ex:statement1 rdf:object ex:Carol .

# Et enfin l'annotation :
ex:statement1 ex:assertedBy ex:Alice .
```

Cela represente **5 triplets supplementaires** pour annoter un seul fait. De plus, la reification ne **garantit pas** que le triplet original existe reellement dans le graphe.

Voyons cela en pratique avec rdflib : la reification classique est verbeuse et fragile.

In [ ]:
from rdflib import Graph, URIRef, Literal, Namespace, BNode
from rdflib.namespace import RDF, RDFS, FOAF, XSD

EX = Namespace("http://example.org/")

g_reif = Graph()
g_reif.bind("ex", EX)
g_reif.bind("foaf", FOAF)

# Le fait original
g_reif.add((EX.Bob, FOAF.knows, EX.Carol))

# Reification classique : decomposer le triplet
stmt = EX.statement1
g_reif.add((stmt, RDF.type, RDF.Statement))
g_reif.add((stmt, RDF.subject, EX.Bob))
g_reif.add((stmt, RDF.predicate, FOAF.knows))
g_reif.add((stmt, RDF.object, EX.Carol))

# Annotations : qui affirme ce fait, avec quel degre de confiance ?
g_reif.add((stmt, EX.assertedBy, EX.Alice))
g_reif.add((stmt, EX.confidence, Literal(0.85, datatype=XSD.double)))

print(f"Nombre total de triplets : {len(g_reif)}")
print(f"  - Fait original : 1 triplet")
print(f"  - Reification : 4 triplets")
print(f"  - Annotations : 2 triplets")
print(f"  - Total : 7 triplets pour annoter 1 fait")
print()
print(g_reif.serialize(format="turtle"))

### Interpretation : cout de la reification

| Approche | Triplets necessaires | Lisibilite | Garantie d'existence |
|----------|---------------------|------------|---------------------|
| Fait seul | 1 | Excellente | Oui |
| Reification classique | 1 + 4 + N annotations | Faible | Non (le Statement est independant) |
| RDF-Star (quoted triple) | 1 + N annotations | Bonne | Configurable |

**Problemes de la reification classique** :
1. **Verbeux** : 4 triplets par assertion reifiee, explosion combinatoire
2. **Deconnecte** : le `rdf:Statement` ne garantit pas que le triplet original existe
3. **Requetes complexes** : les requetes SPARQL pour retrouver les annotations sont longues
4. **Pas standard dans la pratique** : peu de triple stores optimisent la reification

> **Point cle** : RDF-Star (devenu RDF 1.2) resout ces problemes en permettant d'utiliser un triplet directement comme sujet ou objet d'un autre triplet.

---

## 2. RDF 1.2 et les Quoted Triples

### Historique

| Date | Evenement |
|------|----------|
| 2014 | Olaf Hartig propose RDF* (RDF-Star) dans un article academique |
| 2017-2019 | Adoption par des triple stores (Blazegraph, Stardog, Jena) |
| 2021 | Le W3C cree le RDF-Star Community Group |
| 2023 | Le W3C integre RDF-Star dans la specification RDF 1.2 |
| 2024 | RDF 1.2 devient une W3C Recommendation (Candidate) |

### Syntaxe des Quoted Triples

Un **quoted triple** (triplet cite) est un triplet RDF utilise comme sujet ou objet d'un autre triplet. En Turtle-Star, la syntaxe utilise des chevrons doubles `<< ... >>` :

```turtle
# Syntaxe RDF-Star / Turtle-Star
<< ex:Bob foaf:knows ex:Carol >> ex:assertedBy ex:Alice .
<< ex:Bob foaf:knows ex:Carol >> ex:confidence 0.85 .
```

Cela remplace les 7 triplets de la reification classique par seulement **2 triplets** (plus le fait original si on veut l'asserter aussi).

### Comparaison visuelle

| Aspect | Reification classique | RDF-Star / RDF 1.2 |
|--------|----------------------|--------------------|
| **Syntaxe** | `ex:stmt rdf:subject ex:Bob .` (4 triplets) | `<< ex:Bob foaf:knows ex:Carol >>` (inline) |
| **Triplets par annotation** | 4 + N | N |
| **Requetes SPARQL** | JOIN sur Statement | Pattern `<< ?s ?p ?o >>` |
| **Support triple stores** | Universel | Croissant (Jena, Blazegraph, Stardog, Oxigraph) |
| **Support rdflib** | Natif (toutes versions) | Version 7.x+ |

### 2.1 Quoted Triples avec rdflib

rdflib 7.x supporte la syntaxe Turtle-Star pour parser et serialiser des quoted triples. Construisons le meme exemple que la reification classique, mais avec RDF-Star.

In [ ]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, FOAF, XSD

EX = Namespace("http://example.org/")

# Methode : parser du Turtle-Star
turtle_star_data = """
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

# Le fait original
ex:Bob foaf:knows ex:Carol .

# Annotations via quoted triples (RDF-Star)
<< ex:Bob foaf:knows ex:Carol >> ex:assertedBy ex:Alice .
<< ex:Bob foaf:knows ex:Carol >> ex:confidence "0.85"^^xsd:double .
<< ex:Bob foaf:knows ex:Carol >> ex:since "2020-01-15"^^xsd:date .
"""

g_star = Graph()
g_star.bind("ex", EX)
g_star.bind("foaf", FOAF)

try:
    g_star.parse(data=turtle_star_data, format="turtle")
    print(f"Triplets charges (RDF-Star) : {len(g_star)}")
    print()
    print("=== Serialisation Turtle-Star ===")
    print(g_star.serialize(format="turtle"))
except Exception as e:
    print(f"Erreur lors du parsing Turtle-Star : {type(e).__name__}: {e}")
    print()
    print("Note : Le support RDF-Star dans rdflib depend de la version.")
    print("Verifiez que rdflib >= 7.0 est installe.")

### Interpretation

Avec RDF-Star, les annotations sont directement attachees au triplet cite `<< ex:Bob foaf:knows ex:Carol >>`. Comparons le nombre de triplets :

| Approche | Triplets pour "Bob connait Carol" | Triplets pour 3 annotations | Total |
|----------|----------------------------------|----------------------------|-------|
| Reification classique | 1 + 4 (Statement) | 3 | **8** |
| RDF-Star | 1 | 3 | **4** |

La reduction est significative : 50% de triplets en moins pour un cas simple. L'ecart se creuse avec le nombre d'annotations.

> **Note technique** : Un quoted triple n'est pas necessairement asserte dans le graphe. `<< ex:Bob foaf:knows ex:Carol >> ex:assertedBy ex:Alice .` ne signifie pas automatiquement que Bob connait Carol -- seulement qu'Alice le dit. Pour asserter le fait, il faut aussi ajouter `ex:Bob foaf:knows ex:Carol .` explicitement.

---

## 3. Construction programmatique de Quoted Triples

Voyons comment construire des quoted triples par programmation avec rdflib, sans passer par le parsing de Turtle-Star.

### 3.1 API rdflib pour les Quoted Triples

rdflib 7.x fournit la classe `rdflib.term.Triple` qui encapsule un triplet (sujet, predicat, objet) en un seul terme pouvant servir de sujet ou d'objet dans un autre triplet.

In [ ]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, FOAF, XSD

EX = Namespace("http://example.org/")

g_prog = Graph()
g_prog.bind("ex", EX)
g_prog.bind("foaf", FOAF)

try:
    # rdflib 7.x : utiliser rdflib.term.Triple
    from rdflib.term import Triple as QuotedTriple
    
    # Creer le quoted triple
    qt = QuotedTriple(EX.Bob, FOAF.knows, EX.Carol)
    
    # Asserter le fait original
    g_prog.add((EX.Bob, FOAF.knows, EX.Carol))
    
    # Ajouter des annotations sur le quoted triple
    g_prog.add((qt, EX.assertedBy, EX.Alice))
    g_prog.add((qt, EX.confidence, Literal(0.85, datatype=XSD.double)))
    g_prog.add((qt, EX.source, Literal("Interview 2024-03-15")))
    
    print(f"Graphe construit : {len(g_prog)} triplets")
    print(f"  Type du quoted triple : {type(qt).__name__}")
    print()
    print("=== Serialisation Turtle-Star ===")
    print(g_prog.serialize(format="turtle"))
    
except ImportError:
    print("La classe Triple (QuotedTriple) n'est pas disponible dans cette version de rdflib.")
    print("Alternative : construire via parsing Turtle-Star.")
    print()
    
    ttl = """
    @prefix ex: <http://example.org/> .
    @prefix foaf: <http://xmlns.com/foaf/0.1/> .
    @prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
    
    ex:Bob foaf:knows ex:Carol .
    << ex:Bob foaf:knows ex:Carol >> ex:assertedBy ex:Alice .
    << ex:Bob foaf:knows ex:Carol >> ex:confidence "0.85"^^xsd:double .
    << ex:Bob foaf:knows ex:Carol >> ex:source "Interview 2024-03-15" .
    """
    g_prog.parse(data=ttl, format="turtle")
    print(f"Graphe construit (via parsing) : {len(g_prog)} triplets")
    print(g_prog.serialize(format="turtle"))

### Interpretation

La construction programmatique suit le meme schema que l'ajout de triplets classiques, mais le sujet est un `QuotedTriple` au lieu d'un `URIRef` :

| Operation | RDF classique | RDF-Star |
|-----------|--------------|----------|
| Sujet | `URIRef("http://...")` | `QuotedTriple(s, p, o)` |
| Ajout | `g.add((uri, pred, obj))` | `g.add((qt, pred, obj))` |
| Serialisation | Turtle standard | Turtle-Star (`<< ... >>`) |

> **Conseil** : Si votre version de rdflib ne supporte pas `rdflib.term.Triple`, utilisez le parsing de Turtle-Star comme alternative. Le resultat est identique.

### 3.2 Quoted Triples imbriques

RDF-Star permet l'imbrication : un quoted triple peut lui-meme contenir un quoted triple. Par exemple : "Alice rapporte que Bob croit que Carol connait Dave".

```
Alice rapporte:
  +-- Bob croit:
      +-- Carol connait Dave
```

Construisons cet exemple.

In [ ]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import FOAF, XSD

EX = Namespace("http://example.org/")

nested_ttl = """
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

# Niveau 1 : Carol connait Dave (fait de base)
ex:Carol foaf:knows ex:Dave .

# Niveau 2 : Bob croit ce fait
<< ex:Carol foaf:knows ex:Dave >> ex:believedBy ex:Bob .

# Niveau 3 : Alice rapporte la croyance de Bob
<< << ex:Carol foaf:knows ex:Dave >> ex:believedBy ex:Bob >> ex:reportedBy ex:Alice .
<< << ex:Carol foaf:knows ex:Dave >> ex:believedBy ex:Bob >> ex:reportDate "2024-06-01"^^xsd:date .
"""

g_nested = Graph()
g_nested.bind("ex", EX)
g_nested.bind("foaf", FOAF)

try:
    g_nested.parse(data=nested_ttl, format="turtle")
    print(f"Triplets charges : {len(g_nested)}")
    print()
    print("=== Serialisation Turtle-Star (imbrication) ===")
    print(g_nested.serialize(format="turtle"))
except Exception as e:
    print(f"Erreur : {e}")
    print("L'imbrication de quoted triples necessite rdflib >= 7.0.")

### Interpretation : imbrication

Chaque niveau d'imbrication ajoute du contexte sans modifier le fait de base :

| Niveau | Triplet | Signification |
|--------|---------|---------------|
| 1 | `Carol foaf:knows Dave` | Fait asserte |
| 2 | `<< Carol knows Dave >> believedBy Bob` | Bob croit ce fait |
| 3 | `<< << Carol knows Dave >> believedBy Bob >> reportedBy Alice` | Alice rapporte la croyance de Bob |

> **Attention** : L'imbrication profonde (>2 niveaux) rend les requetes SPARQL-Star complexes. En pratique, un seul niveau d'imbrication couvre la majorite des cas d'usage.

---

## 4. SPARQL-Star : interroger les Quoted Triples

SPARQL-Star est l'extension de SPARQL qui permet de requeter des graphes contenant des quoted triples. La syntaxe utilise les memes chevrons doubles `<< ... >>` dans les patterns.

### Syntaxe SPARQL-Star

| Pattern | Description |
|---------|-------------|
| `<< ?s ?p ?o >> ?annPred ?annObj .` | Trouver les annotations d'un triplet |
| `<< ex:Bob foaf:knows ?who >> ?p ?o .` | Annotations d'un triplet specifique |
| `BIND(<< ?s ?p ?o >> AS ?qt)` | Capturer un quoted triple dans une variable |

### 4.1 Construire un graphe de connaissances annote

Creons un graphe plus riche avec des relations annotees : provenance, confiance et dates.

In [ ]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib.namespace import RDF, FOAF, XSD

EX = Namespace("http://example.org/")

kg_ttl = """
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

# === Personnes ===
ex:Alice a foaf:Person ; foaf:name "Alice Dupont" .
ex:Bob a foaf:Person ; foaf:name "Bob Martin" .
ex:Carol a foaf:Person ; foaf:name "Carol Laurent" .
ex:Dave a foaf:Person ; foaf:name "Dave Petit" .

# === Faits avec annotations RDF-Star ===

# Fait 1 : Alice connait Bob (haute confiance, source LinkedIn)
ex:Alice foaf:knows ex:Bob .
<< ex:Alice foaf:knows ex:Bob >> ex:confidence "0.95"^^xsd:double .
<< ex:Alice foaf:knows ex:Bob >> ex:source "LinkedIn" .
<< ex:Alice foaf:knows ex:Bob >> ex:since "2019-03-01"^^xsd:date .

# Fait 2 : Bob connait Carol (confiance moyenne, source Twitter)
ex:Bob foaf:knows ex:Carol .
<< ex:Bob foaf:knows ex:Carol >> ex:confidence "0.60"^^xsd:double .
<< ex:Bob foaf:knows ex:Carol >> ex:source "Twitter" .
<< ex:Bob foaf:knows ex:Carol >> ex:since "2021-07-15"^^xsd:date .

# Fait 3 : Carol connait Dave (basse confiance, source rumeur)
ex:Carol foaf:knows ex:Dave .
<< ex:Carol foaf:knows ex:Dave >> ex:confidence "0.30"^^xsd:double .
<< ex:Carol foaf:knows ex:Dave >> ex:source "Rumeur" .

# Fait 4 : Alice connait Dave (non asserte, seulement affirme par Bob)
<< ex:Alice foaf:knows ex:Dave >> ex:assertedBy ex:Bob .
<< ex:Alice foaf:knows ex:Dave >> ex:confidence "0.50"^^xsd:double .
"""

g_kg = Graph()
g_kg.bind("ex", EX)
g_kg.bind("foaf", FOAF)

try:
    g_kg.parse(data=kg_ttl, format="turtle")
    print(f"Graphe de connaissances : {len(g_kg)} triplets")
    print()
    print("=== Serialisation ===")
    print(g_kg.serialize(format="turtle"))
except Exception as e:
    print(f"Erreur : {e}")
    print("Verifiez que rdflib >= 7.0 est installe.")

### Interpretation : graphe de connaissances annote

Ce graphe illustre un reseau social avec des annotations de qualite :

| Relation | Confiance | Source | Depuis | Asserte ? |
|----------|-----------|--------|--------|-----------|
| Alice connait Bob | 0.95 | LinkedIn | 2019-03-01 | Oui |
| Bob connait Carol | 0.60 | Twitter | 2021-07-15 | Oui |
| Carol connait Dave | 0.30 | Rumeur | - | Oui |
| Alice connait Dave | 0.50 | - | - | **Non** (seulement affirme par Bob) |

> **Point cle** : Le fait 4 n'est **pas asserte** dans le graphe (`ex:Alice foaf:knows ex:Dave .` n'apparait pas). Seul le quoted triple et ses annotations existent. Cela signifie que le graphe ne dit pas qu'Alice connait Dave -- il dit seulement que Bob le pretend.

### 4.2 Requetes SPARQL-Star

Interrogeons le graphe avec SPARQL-Star pour extraire les annotations.

In [ ]:
# Requete 1 : Toutes les relations annotees avec confiance
query_all = """
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?person1 ?person2 ?confidence ?source ?since
WHERE {
    << ?person1 foaf:knows ?person2 >> ex:confidence ?confidence .
    OPTIONAL { << ?person1 foaf:knows ?person2 >> ex:source ?source . }
    OPTIONAL { << ?person1 foaf:knows ?person2 >> ex:since ?since . }
}
ORDER BY DESC(?confidence)
"""

try:
    results = g_kg.query(query_all)
    
    print("=== Relations annotees (triees par confiance) ===")
    print(f"{'Personne 1':<15} {'Personne 2':<15} {'Confiance':>10} {'Source':<12} {'Depuis'}")
    print("-" * 70)
    for row in results:
        p1 = str(row.person1).split("/")[-1]
        p2 = str(row.person2).split("/")[-1]
        conf = f"{float(row.confidence):.2f}"
        src = str(row.source) if row.source else "-"
        since = str(row.since) if row.since else "-"
        print(f"{p1:<15} {p2:<15} {conf:>10} {src:<12} {since}")
except Exception as e:
    print(f"Erreur SPARQL-Star : {e}")
    print("Le moteur SPARQL de rdflib ne supporte peut-etre pas la syntaxe SPARQL-Star.")

### 4.3 Filtrer par score de confiance

Un cas d'usage important : ne retenir que les relations ayant un score de confiance superieur a un seuil. Cela permet de construire une vue "fiable" du graphe.

In [ ]:
# Requete : Relations a haute confiance (>= 0.60)
query_high = """
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?person1 ?person2 ?confidence ?source
WHERE {
    << ?person1 foaf:knows ?person2 >> ex:confidence ?confidence .
    FILTER (?confidence >= 0.60)
    OPTIONAL { << ?person1 foaf:knows ?person2 >> ex:source ?source . }
}
ORDER BY DESC(?confidence)
"""

# Requete : Relations a basse confiance (< 0.60)
query_low = """
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?person1 ?person2 ?confidence ?source
WHERE {
    << ?person1 foaf:knows ?person2 >> ex:confidence ?confidence .
    FILTER (?confidence < 0.60)
    OPTIONAL { << ?person1 foaf:knows ?person2 >> ex:source ?source . }
}
ORDER BY ?confidence
"""

try:
    print("=== Relations FIABLES (confiance >= 0.60) ===")
    for row in g_kg.query(query_high):
        p1 = str(row.person1).split("/")[-1]
        p2 = str(row.person2).split("/")[-1]
        src = str(row.source) if row.source else "-"
        print(f"  {p1} connait {p2} (confiance={float(row.confidence):.2f}, source={src})")
    
    print()
    print("=== Relations INCERTAINES (confiance < 0.60) ===")
    for row in g_kg.query(query_low):
        p1 = str(row.person1).split("/")[-1]
        p2 = str(row.person2).split("/")[-1]
        src = str(row.source) if row.source else "-"
        print(f"  {p1} connait {p2} (confiance={float(row.confidence):.2f}, source={src})")
except Exception as e:
    print(f"Erreur : {e}")

### Interpretation : filtrage par confiance

Le filtrage separe les faits fiables des faits incertains :

| Categorie | Relations | Usage recommande |
|-----------|-----------|------------------|
| Haute confiance (>= 0.60) | Alice-Bob, Bob-Carol | Utiliser pour le raisonnement |
| Basse confiance (< 0.60) | Carol-Dave, Alice-Dave | A verifier, ne pas propager |

Ce type de filtrage est essentiel dans les **graphes de connaissances** : on peut construire une vue "fiable" du graphe en excluant les assertions douteuses.

> **Application concrete** : Dans un systeme de recommandation, on ne recommanderait un contact qu'a partir de relations de confiance >= 0.70.

---

## 5. Cas d'usage concrets

RDF-Star trouve ses applications dans de nombreux domaines. Explorons trois cas d'usage representatifs.

### 5.1 Provenance et attribution

Le cas d'usage le plus naturel : enregistrer **qui** a produit une information, **quand**, et avec **quelle methode**. Ce pattern s'integre naturellement avec le vocabulaire PROV-O du W3C.

In [ ]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS, XSD

EX = Namespace("http://example.org/")
PROV = Namespace("http://www.w3.org/ns/prov#")
SDO = Namespace("https://schema.org/")

prov_ttl = """
@prefix ex: <http://example.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

# Fait : Paris a une population de 2,1 millions
ex:Paris schema:population "2161000"^^xsd:integer .

# Provenance du fait
<< ex:Paris schema:population "2161000"^^xsd:integer >>
    prov:wasDerivedFrom ex:INSEE_2023 ;
    prov:generatedAtTime "2023-12-01"^^xsd:date ;
    ex:method "Recensement" ;
    ex:accuracy "haute" .

# Fait : Paris a une superficie de 105.4 km2
ex:Paris schema:area "105.4"^^xsd:double .

<< ex:Paris schema:area "105.4"^^xsd:double >>
    prov:wasDerivedFrom ex:IGN_2023 ;
    prov:generatedAtTime "2023-06-15"^^xsd:date ;
    ex:method "Mesure cadastrale" .
"""

g_prov = Graph()
g_prov.bind("ex", EX)
g_prov.bind("prov", PROV)
g_prov.bind("schema", SDO)

try:
    g_prov.parse(data=prov_ttl, format="turtle")
    print(f"Graphe de provenance : {len(g_prov)} triplets")
    print()
    
    # Requete : quels faits, avec quelle source ?
    query_prov = """
    PREFIX ex: <http://example.org/>
    PREFIX prov: <http://www.w3.org/ns/prov#>
    PREFIX schema: <https://schema.org/>
    
    SELECT ?subject ?property ?value ?source ?date ?method
    WHERE {
        << ?subject ?property ?value >> prov:wasDerivedFrom ?source .
        OPTIONAL { << ?subject ?property ?value >> prov:generatedAtTime ?date . }
        OPTIONAL { << ?subject ?property ?value >> ex:method ?method . }
    }
    """
    
    print("=== Faits avec provenance ===")
    print(f"{'Sujet':<10} {'Propriete':<20} {'Valeur':<12} {'Source':<15} {'Date':<12} {'Methode'}")
    print("-" * 85)
    for row in g_prov.query(query_prov):
        subj = str(row.subject).split("/")[-1]
        prop = str(row.property).split("/")[-1]
        src = str(row.source).split("/")[-1]
        date = str(row.date) if row.date else "-"
        method = str(row.method) if row.method else "-"
        print(f"{subj:<10} {prop:<20} {str(row.value):<12} {src:<15} {date:<12} {method}")

except Exception as e:
    print(f"Erreur : {e}")

### Interpretation : provenance

| Fait | Source | Date | Methode |
|------|--------|------|---------|
| Population de Paris = 2,161,000 | INSEE 2023 | 2023-12-01 | Recensement |
| Superficie de Paris = 105.4 km2 | IGN 2023 | 2023-06-15 | Mesure cadastrale |

**Avantage de RDF-Star** : la provenance est directement rattachee au fait qu'elle concerne, sans intermediaire. En reification classique, il aurait fallu 2 x 4 = 8 triplets supplementaires pour les Statements seuls.

### 5.2 Annotations temporelles

RDF-Star est ideal pour exprimer la validite temporelle des faits : un emploi, un statut, une relation qui change au fil du temps.

In [ ]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, XSD

EX = Namespace("http://example.org/")
SDO = Namespace("https://schema.org/")

temporal_ttl = """
@prefix ex: <http://example.org/> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

# Personnes et organisations
ex:Marie a schema:Person ; schema:name "Marie" .
ex:Sorbonne a schema:Organization ; schema:name "Sorbonne" .
ex:CNRS a schema:Organization ; schema:name "CNRS" .
ex:MIT a schema:Organization ; schema:name "MIT" .

# Emploi actuel
ex:Marie schema:worksFor ex:Sorbonne .
<< ex:Marie schema:worksFor ex:Sorbonne >> ex:startDate "2020-09-01"^^xsd:date .
<< ex:Marie schema:worksFor ex:Sorbonne >> ex:role "Professeure" .
<< ex:Marie schema:worksFor ex:Sorbonne >> ex:status "actif" .

# Emploi precedent 1
<< ex:Marie schema:worksFor ex:CNRS >> ex:startDate "2015-01-15"^^xsd:date .
<< ex:Marie schema:worksFor ex:CNRS >> ex:endDate "2020-08-31"^^xsd:date .
<< ex:Marie schema:worksFor ex:CNRS >> ex:role "Chercheuse" .
<< ex:Marie schema:worksFor ex:CNRS >> ex:status "termine" .

# Emploi precedent 2
<< ex:Marie schema:worksFor ex:MIT >> ex:startDate "2012-09-01"^^xsd:date .
<< ex:Marie schema:worksFor ex:MIT >> ex:endDate "2014-12-31"^^xsd:date .
<< ex:Marie schema:worksFor ex:MIT >> ex:role "Post-doc" .
<< ex:Marie schema:worksFor ex:MIT >> ex:status "termine" .
"""

g_temporal = Graph()
g_temporal.bind("ex", EX)
g_temporal.bind("schema", SDO)

try:
    g_temporal.parse(data=temporal_ttl, format="turtle")
    print(f"Graphe temporel : {len(g_temporal)} triplets")
    print()
    
    # Requete : parcours professionnel chronologique
    query_career = """
    PREFIX ex: <http://example.org/>
    PREFIX schema: <https://schema.org/>
    
    SELECT ?orgName ?role ?startDate ?endDate ?status
    WHERE {
        << ex:Marie schema:worksFor ?org >> ex:role ?role .
        << ex:Marie schema:worksFor ?org >> ex:startDate ?startDate .
        << ex:Marie schema:worksFor ?org >> ex:status ?status .
        OPTIONAL { << ex:Marie schema:worksFor ?org >> ex:endDate ?endDate . }
        ?org schema:name ?orgName .
    }
    ORDER BY ?startDate
    """
    
    print("=== Parcours professionnel de Marie ===")
    print(f"{'Organisation':<15} {'Role':<15} {'Debut':<12} {'Fin':<12} {'Statut'}")
    print("-" * 65)
    for row in g_temporal.query(query_career):
        end = str(row.endDate) if row.endDate else "en cours"
        print(f"{str(row.orgName):<15} {str(row.role):<15} {str(row.startDate):<12} {end:<12} {str(row.status)}")

except Exception as e:
    print(f"Erreur : {e}")

### Interpretation : annotations temporelles

Le parcours professionnel est represente de maniere elegante :

| Periode | Organisation | Role | Statut |
|---------|-------------|------|--------|
| 2012-2014 | MIT | Post-doc | Termine |
| 2015-2020 | CNRS | Chercheuse | Termine |
| 2020-... | Sorbonne | Professeure | Actif |

**Sans RDF-Star**, cet historique necessiterait soit :
- Des **blank nodes intermediaires** (ex: `ex:emploi1 ex:employer ex:MIT ; ex:role "Post-doc" .`) ce qui perd le lien direct `Marie -> worksFor -> MIT`
- De la **reification classique** (4 triplets par relation + annotations)

Avec RDF-Star, le fait principal est directement annotable et le graphe reste navigable comme un graphe RDF classique.

### 5.3 Fusion multi-sources avec scores de confiance

Dans un graphe de connaissances construit a partir de sources multiples, chaque source peut affirmer des faits contradictoires. RDF-Star permet de conserver toutes les versions et de les departager.

In [ ]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, XSD

EX = Namespace("http://example.org/")
SDO = Namespace("https://schema.org/")

# Deux sources donnent des populations differentes pour Lyon
multi_ttl = """
@prefix ex: <http://example.org/> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Lyon a schema:City ; schema:name "Lyon" .

# Source 1 : INSEE (haute confiance)
<< ex:Lyon schema:population "522250"^^xsd:integer >>
    ex:source "INSEE" ;
    ex:confidence "0.95"^^xsd:double ;
    ex:year "2023"^^xsd:gYear .

# Source 2 : Wikipedia (confiance moyenne)
<< ex:Lyon schema:population "516092"^^xsd:integer >>
    ex:source "Wikipedia" ;
    ex:confidence "0.70"^^xsd:double ;
    ex:year "2021"^^xsd:gYear .

# Source 3 : Estimation locale (basse confiance)
<< ex:Lyon schema:population "530000"^^xsd:integer >>
    ex:source "Estimation mairie" ;
    ex:confidence "0.40"^^xsd:double ;
    ex:year "2024"^^xsd:gYear .
"""

g_multi = Graph()
g_multi.bind("ex", EX)
g_multi.bind("schema", SDO)

try:
    g_multi.parse(data=multi_ttl, format="turtle")
    print(f"Graphe multi-sources : {len(g_multi)} triplets")
    print()
    
    # Requete : toutes les valeurs avec leurs sources
    query_sources = """
    PREFIX ex: <http://example.org/>
    PREFIX schema: <https://schema.org/>
    
    SELECT ?pop ?source ?confidence ?year
    WHERE {
        << ex:Lyon schema:population ?pop >> ex:source ?source .
        << ex:Lyon schema:population ?pop >> ex:confidence ?confidence .
        << ex:Lyon schema:population ?pop >> ex:year ?year .
    }
    ORDER BY DESC(?confidence)
    """
    
    print("=== Population de Lyon selon differentes sources ===")
    print(f"{'Source':<20} {'Population':>12} {'Confiance':>10} {'Annee':>6}")
    print("-" * 52)
    for row in g_multi.query(query_sources):
        pop = f"{int(row.pop):,d}"
        conf = f"{float(row.confidence):.2f}"
        print(f"{str(row.source):<20} {pop:>12} {conf:>10} {str(row.year):>6}")
    
    # Valeur la plus fiable
    query_best = """
    PREFIX ex: <http://example.org/>
    PREFIX schema: <https://schema.org/>
    
    SELECT ?pop ?source ?confidence
    WHERE {
        << ex:Lyon schema:population ?pop >> ex:confidence ?confidence .
        << ex:Lyon schema:population ?pop >> ex:source ?source .
    }
    ORDER BY DESC(?confidence)
    LIMIT 1
    """
    
    print()
    for row in g_multi.query(query_best):
        print(f"Valeur retenue (confiance max) : {int(row.pop):,d} (source: {row.source})")

except Exception as e:
    print(f"Erreur : {e}")

### Interpretation : fusion multi-sources

Ce pattern est fondamental pour les graphes de connaissances industriels :

1. **Conserver toutes les valeurs** : chaque source a sa propre estimation
2. **Annoter la confiance** : permettre un filtrage par fiabilite
3. **Tracer la provenance** : savoir d'ou vient chaque information
4. **Selectionner la meilleure** : prendre la valeur la plus fiable pour un usage donne

| Strategie de selection | Requete | Usage |
|----------------------|---------|-------|
| Plus haute confiance | `ORDER BY DESC(?confidence) LIMIT 1` | Vue par defaut |
| Plus recente | `ORDER BY DESC(?year) LIMIT 1` | Donnees a jour |
| Moyenne ponderee | Calcul hors SPARQL | Estimation consolidee |

---

## 6. Serialisation et interoperabilite

RDF-Star peut etre serialise dans plusieurs formats. Tous ne supportent pas encore la syntaxe nativement.

### Formats supportant RDF-Star

| Format | Support RDF-Star | Syntaxe |
|--------|-----------------|--------|
| **Turtle-Star** | Natif | `<< s p o >> annPred annObj .` |
| **N-Triples-Star** | Natif | `<< <s> <p> <o> >> <annPred> <annObj> .` |
| **TriG-Star** | Natif | Comme Turtle-Star, avec graphes nommes |
| **JSON-LD-Star** | En discussion | Utilisation de `@annotation` (pas encore standardise) |
| **RDF/XML** | Non supporte | Pas de syntaxe prevue |

### Serialisation dans differents formats

Testons les formats disponibles dans rdflib pour un graphe contenant des quoted triples.

In [ ]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import FOAF, XSD

EX = Namespace("http://example.org/")

simple_star = """
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Alice foaf:knows ex:Bob .
<< ex:Alice foaf:knows ex:Bob >> ex:confidence "0.9"^^xsd:double .
"""

g_ser = Graph()
g_ser.bind("ex", EX)
g_ser.bind("foaf", FOAF)

try:
    g_ser.parse(data=simple_star, format="turtle")
    
    formats_to_test = [
        ("Turtle (Turtle-Star)", "turtle"),
        ("N-Triples (NT-Star)", "nt"),
    ]
    
    for label, fmt in formats_to_test:
        try:
            output = g_ser.serialize(format=fmt)
            print(f"=== {label} ===")
            print(output.strip())
            print()
        except Exception as e:
            print(f"=== {label} : Non supporte ===")
            print(f"  Erreur : {e}")
            print()
    
    # JSON-LD
    try:
        jsonld_out = g_ser.serialize(format="json-ld", indent=2)
        print("=== JSON-LD ===")
        print(jsonld_out.strip())
    except Exception as e:
        print("=== JSON-LD : Support partiel ===")
        print("  Les quoted triples ne sont pas encore standardises en JSON-LD.")
        print(f"  Erreur : {e}")

except Exception as e:
    print(f"Erreur de parsing : {e}")

### Interpretation : formats de serialisation

- **Turtle-Star** et **N-Triples-Star** sont bien supportes par rdflib 7.x
- **JSON-LD-Star** est en cours de discussion au W3C (pas encore standardise)
- **RDF/XML** ne supportera probablement jamais RDF-Star (syntaxe XML non adaptee)

> **Conseil pratique** : Utilisez Turtle-Star pour le stockage et les echanges avec des triple stores compatibles. Pour les APIs web, attendez la standardisation de JSON-LD-Star ou convertissez en reification classique si necessaire.

---

## 7. Exercices

### Exercice 1 : Creer des Quoted Triples pour des critiques de film

Creez un graphe RDF-Star representant les avis de trois critiques sur un film :
- Critique A : note 4/5, source "Le Monde", date 2024-01-15
- Critique B : note 3/5, source "Telerama", date 2024-01-20
- Critique C : note 5/5, source "Cahiers du Cinema", date 2024-02-01

Le fait principal est `ex:Film1 ex:rating ?note` avec les annotations de provenance.

**Indice** : Chaque critique produit un quoted triple different car la note differe.

In [ ]:
# Exercice 1 : Critiques de film avec RDF-Star
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, XSD

EX = Namespace("http://example.org/")

# TODO : Completez le Turtle-Star ci-dessous
film_ttl = """
@prefix ex: <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Film1 a ex:Film ; ex:title "Les Temps Modernes" .

# TODO : Ajoutez les 3 critiques avec quoted triples
# Critique A : note 4/5
# << ex:Film1 ex:rating "4"^^xsd:integer >> ex:reviewer "Critique A" .
# << ex:Film1 ex:rating "4"^^xsd:integer >> ex:source "Le Monde" .
# << ex:Film1 ex:rating "4"^^xsd:integer >> ex:date "2024-01-15"^^xsd:date .

# Critique B : note 3/5
# ...

# Critique C : note 5/5
# ...
"""

g_ex1 = Graph()
g_ex1.bind("ex", EX)

# TODO : Parsez et affichez
# g_ex1.parse(data=film_ttl, format="turtle")
# print(g_ex1.serialize(format="turtle"))

# TODO : Requete SPARQL-Star pour lister les critiques
# query = """
# PREFIX ex: <http://example.org/>
# SELECT ?note ?reviewer ?source ?date
# WHERE {
#     << ex:Film1 ex:rating ?note >> ex:reviewer ?reviewer .
#     << ex:Film1 ex:rating ?note >> ex:source ?source .
#     << ex:Film1 ex:rating ?note >> ex:date ?date .
# }
# ORDER BY ?date
# """
# for row in g_ex1.query(query):
#     print(f"{row.reviewer} ({row.source}) : {row.note}/5 le {row.date}")

print("Completez l'exercice et decommentez le code.")

### Exercice 2 : Requeter avec SPARQL-Star

En utilisant le graphe de connaissances annote de la section 4.1 (variable `g_kg`), ecrivez des requetes SPARQL-Star pour :

1. Trouver toutes les relations **assertees par une tierce personne** (c'est-a-dire qui ont un `ex:assertedBy`)
2. Trouver les relations **etablies depuis plus de 3 ans** (avant 2022-01-01)

**Indice** : Pour la requete 2, utilisez `FILTER(?since < "2022-01-01"^^xsd:date)`

In [ ]:
# Exercice 2 : Requetes SPARQL-Star avancees

# Requete 2a : Relations assertees par une tierce personne
query_2a = """
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

# TODO : Trouvez les relations avec ex:assertedBy
SELECT ?person1 ?person2 ?asserter
WHERE {
    # << ?person1 foaf:knows ?person2 >> ex:assertedBy ?asserter .
}
"""

# Requete 2b : Relations anciennes (avant 2022)
query_2b = """
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

# TODO : Trouvez les relations anterieures a 2022
SELECT ?person1 ?person2 ?since
WHERE {
    # << ?person1 foaf:knows ?person2 >> ex:since ?since .
    # FILTER(?since < "2022-01-01"^^xsd:date)
}
"""

# TODO : Decommentez et executez les requetes sur g_kg
# print("=== 2a : Relations assertees par un tiers ===")
# for row in g_kg.query(query_2a):
#     p1 = str(row.person1).split("/")[-1]
#     p2 = str(row.person2).split("/")[-1]
#     asserter = str(row.asserter).split("/")[-1]
#     print(f"  {asserter} affirme : {p1} connait {p2}")

# print("\n=== 2b : Relations anterieures a 2022 ===")
# for row in g_kg.query(query_2b):
#     p1 = str(row.person1).split("/")[-1]
#     p2 = str(row.person2).split("/")[-1]
#     print(f"  {p1} connait {p2} depuis {row.since}")

print("Completez les requetes et decommentez le code.")

---

## 8. Ecosysteme et compatibilite

RDF-Star est de plus en plus adopte par l'ecosysteme du Web Semantique.

### Support dans les triple stores

| Triple Store | Support RDF-Star | SPARQL-Star | Notes |
|-------------|-----------------|-------------|-------|
| **Apache Jena** (Fuseki) | Oui (depuis 4.2) | Oui | Implementation de reference |
| **Blazegraph** | Oui (extensions) | Oui | Support historique |
| **Stardog** | Oui (depuis 7.4) | Oui | Support commercial complet |
| **Oxigraph** | Oui (depuis 0.3) | Oui | Store Rust performant |
| **GraphDB** (Ontotext) | Oui (depuis 10.0) | Oui | Support entreprise |
| **Amazon Neptune** | Oui (depuis 2023) | Oui | Service cloud |
| **Virtuoso** | Partiel | Partiel | Via extensions |

### Support dans les bibliotheques

| Bibliotheque | Langage | Support | Version minimum |
|-------------|---------|---------|----------------|
| **rdflib** | Python | Oui | 7.0 |
| **Apache Jena** | Java | Oui | 4.2 |
| **dotNetRDF** | C# | Partiel | 3.x (en cours) |
| **N3.js** | JavaScript | Oui | 1.16 |
| **RDF4J** | Java | Oui | 4.0 |

### Standardisation W3C

| Document | Statut (2024) |
|----------|---------------|
| RDF 1.2 Concepts | Candidate Recommendation |
| RDF 1.2 Turtle | Candidate Recommendation |
| RDF 1.2 N-Triples | Candidate Recommendation |
| SPARQL 1.2 Query | Working Draft |
| RDF 1.2 JSON-LD | En discussion |

---

## Resume

### Concepts cles

| Concept | Description |
|---------|-------------|
| **Reification classique** | Mecanisme RDF 1.0 avec `rdf:Statement` (4 triplets par assertion, verbeux) |
| **Quoted Triple** | Triplet utilise comme sujet/objet : `<< s p o >>` (RDF-Star / RDF 1.2) |
| **SPARQL-Star** | Extension SPARQL pour requeter les quoted triples |
| **Provenance** | Annoter qui a dit quoi, quand, avec quelle source |
| **Confiance** | Score de fiabilite attache a une assertion |
| **Annotations temporelles** | Validite dans le temps d'un fait |
| **Fusion multi-sources** | Conserver et departager les valeurs de sources differentes |

### Competences acquises

1. Comprendre les limites de la reification classique et la motivation de RDF-Star
2. Creer des quoted triples avec rdflib (parsing Turtle-Star et API programmatique)
3. Imbriquer des quoted triples pour la provenance multi-niveaux
4. Interroger les annotations avec SPARQL-Star (SELECT, FILTER, OPTIONAL)
5. Appliquer RDF-Star a des cas concrets (provenance, temporalite, multi-sources)

### Pour aller plus loin

- [W3C RDF 1.2 Concepts](https://www.w3.org/TR/rdf12-concepts/) - Specification officielle
- [W3C RDF 1.2 Turtle](https://www.w3.org/TR/rdf12-turtle/) - Syntaxe Turtle-Star
- [SPARQL 1.2 Query](https://www.w3.org/TR/sparql12-query/) - Extension SPARQL-Star
- [Olaf Hartig - RDF* and SPARQL*](https://arxiv.org/abs/1406.3399) - Article fondateur
- [rdflib documentation](https://rdflib.readthedocs.io/) - Guide rdflib

---

Le notebook suivant explore les graphes de connaissances (Knowledge Graphs), en combinant les technologies RDF, SPARQL et les patterns vus dans cette serie.

---

**Navigation** : [<< 10-JSONLD](SW-10-JSONLD.ipynb) | [Index](README.md) | [12-KnowledgeGraphs >>](SW-12-KnowledgeGraphs.ipynb)